In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dailydata=pd.read_csv("/content/drive/MyDrive/Competitions/Predicta/Classify_the_weather/daily_data.csv")
dailydata.tail(10)

In [ ]:
dailydata.shape

(2893, 17)

In [ ]:
submission=pd.read_csv("/content/drive/MyDrive/Competitions/Predicta/Classify_the_weather/submission.csv")
submission.tail(10)

In [ ]:
dailydata.isna().sum()

In [ ]:
def convert_to_numeric(time_str):
    if 'AM' in time_str:
        return int(time_str.split(':')[0])  # Extract hour for AM times
    elif 'PM' in time_str:
        hour = int(time_str.split(':')[0])
        return hour + 12 if hour < 12 else hour  # Convert PM hour to 24-hour format
    else:
        return None  # Handle unexpected cases

# Apply function to 'time' column
dailydata['sunrise'] = dailydata['sunrise'].apply(convert_to_numeric)
dailydata['sunset'] = dailydata['sunset'].apply(convert_to_numeric)

# Print updated DataFrame
print(dailydata)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the specified column
dailydata['temperature_celsius'] = scaler.fit_transform(dailydata[['temperature_celsius']])
dailydata['wind_kph'] = scaler.fit_transform(dailydata[['wind_kph']])
dailydata['wind_degree'] = scaler.fit_transform(dailydata[['wind_degree']])
dailydata['pressure_mb'] = scaler.fit_transform(dailydata[['pressure_mb']])
dailydata['precip_mm'] = scaler.fit_transform(dailydata[['precip_mm']])
dailydata['humidity'] = scaler.fit_transform(dailydata[['humidity']])
dailydata['cloud'] = scaler.fit_transform(dailydata[['cloud']])
dailydata['feels_like_celsius'] = scaler.fit_transform(dailydata[['feels_like_celsius']])
dailydata['visibility_km'] = scaler.fit_transform(dailydata[['visibility_km']])
dailydata['uv_index'] = scaler.fit_transform(dailydata[['uv_index']])
dailydata['gust_kph'] = scaler.fit_transform(dailydata[['gust_kph']])
dailydata['air_quality_us-epa-index'] = scaler.fit_transform(dailydata[['air_quality_us-epa-index']])
dailydata['sunrise'] = scaler.fit_transform(dailydata[['sunrise']])
dailydata['sunset'] = scaler.fit_transform(dailydata[['sunset']])
print(dailydata)

In [ ]:
dailydata=dailydata.drop(["city_id"],axis=1)

In [ ]:
df_to_predict= dailydata[dailydata['condition_text'].isnull()]
df =dailydata[dailydata['condition_text'].notnull()]
df_copy=df
print(df_to_predict)

In [ ]:
unique_values = dailydata['condition_text'].unique()
print(unique_values)

In [ ]:
def encode_labels(df, column_name):

    unique_labels = df[column_name].unique()
    label_to_num = {label: num for num, label in enumerate(unique_labels)}
    num_to_label = {num: label for label, num in label_to_num.items()}

    df[column_name] = df[column_name].map(label_to_num)

    return df, label_to_num, num_to_label

def decode_labels(df, column_name, num_to_label):

    df[column_name] = df[column_name].map(num_to_label)
    return df





In [ ]:
df_copy, label_to_num, num_to_label = encode_labels(df_copy, 'condition_text')
df_copy.tail(20)
# df.shape

In [ ]:
df_1=df_copy.drop(['day_id','condition_text'],axis=1)

In [ ]:
X=df_1
print(X)

In [ ]:
y=df_copy['condition_text']
y

In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature selection using SelectKBest with f_classif (ANOVA F-value between label/feature)
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get selected feature indices
selected_indices = selector.get_support(indices=True)

# Get selected feature names
selected_feature_names = X.columns[selected_indices]
print("Selected features:", selected_feature_names)

# Train a classifier on the selected features
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_selected, y_train)
y_pred = clf.predict(X_test_selected)

# Evaluate accuracy on test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy with selected features: {accuracy * 100:.2f}%')


In [ ]:
X=X[['precip_mm', 'humidity', 'cloud', 'visibility_km',
       'air_quality_us-epa-index']]
print(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# Initialize LightGBM classifier
lgb_clf = lgb.LGBMClassifier(
    random_state=42,
    num_leaves=31,
    learning_rate=0.1,
    n_estimators=100,
    min_child_samples=20,
    reg_alpha=0.1,
    reg_lambda=0.1
)

# Train the model
lgb_clf.fit(X_train, y_train)

# Predictions
y_pred = lgb_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

# Train the model
svm_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 62.50%


In [ ]:
import numpy as np

import xgboost as xgb
from sklearn.metrics import accuracy_score

# Create XGBoost classifier object
xgb_model = xgb.XGBClassifier(objective='multi:softmax',  # Set the objective to multi-class classification
                              num_class=3,  # Number of classes in the dataset
                              eval_metric='merror',  # Evaluation metric (multi-class classification error rate)
                              seed=42)  # Random seed for reproducibility

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on test data
y_pred = xgb_model.predict(X_test)

# Calculate accuracy

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
 pip install catboost

In [ ]:
import numpy as np

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Create CatBoost classifier object
catboost_model = CatBoostClassifier(iterations=1000,  # Number of trees (boosting iterations)
                                    learning_rate=0.1,  # Learning rate
                                    depth=6,  # Depth of trees
                                    loss_function='MultiClass',  # Multi-class classification
                                    eval_metric='Accuracy',  # Metric to use
                                    random_seed=42,  # Random seed for reproducibility
                                    verbose=100  # Print progress every 100 iterations
                                    )

# Train the model
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test))

catboost_model.shrink(114)
# Predict on test data
y_pred = catboost_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')





In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')


In [ ]:
df_2=df_to_predict.drop(['day_id','condition_text'],axis=1)
df_2

In [ ]:
scaler = StandardScaler()

# Fit and transform the specified column
df_2['temperature_celsius'] = scaler.fit_transform(df_2[['temperature_celsius']])
df_2['wind_kph'] = scaler.fit_transform(df_2[['wind_kph']])
df_2['wind_degree'] = scaler.fit_transform(df_2[['wind_degree']])
df_2['pressure_mb'] = scaler.fit_transform(df_2[['pressure_mb']])
df_2['precip_mm'] = scaler.fit_transform(df_2[['precip_mm']])
df_2['humidity'] = scaler.fit_transform(df_2[['humidity']])
df_2['cloud'] = scaler.fit_transform(df_2[['cloud']])
df_2['feels_like_celsius'] = scaler.fit_transform(df_2[['feels_like_celsius']])
df_2['visibility_km'] = scaler.fit_transform(df_2[['visibility_km']])
df_2['uv_index'] = scaler.fit_transform(df_2[['uv_index']])
df_2['gust_kph'] = scaler.fit_transform(df_2[['gust_kph']])
df_2['air_quality_us-epa-index'] = scaler.fit_transform(df_2[['air_quality_us-epa-index']])
df_2['sunrise'] = scaler.fit_transform(df_2[['sunrise']])
df_2['sunset'] = scaler.fit_transform(df_2[['sunset']])
print(df_2)

In [ ]:
df_3=df_2[['precip_mm', 'humidity', 'cloud', 'visibility_km',
       'air_quality_us-epa-index']]
print(df_3)

In [ ]:
y_sol = xgb_model.predict(df_3)

In [ ]:
df_sol= pd.DataFrame(y_sol)
df_sol = df_sol.add_prefix('df2_sol')
print(df_sol)

In [ ]:
# Optionally reset indices to ensure they match
df_to_predict = df_to_predict.drop(columns="condition_text",axis=1)
df_to_predict = df_to_predict.reset_index(drop=True)
df_sol = df_sol.reset_index(drop=True)

# Concatenate the data frames
df_concat = pd.concat([df_to_predict, df_sol], axis=1)
print(df_concat)
df_concat.shape

In [ ]:
df_concat = df_concat.rename(columns={'df2_sol0': 'condition_text'})
print(df_concat)

In [ ]:
# df_concat['condition_text']  = label_encoder.inverse_transform(df_concat['condition_text'])
df_concat = decode_labels(df_concat, 'condition_text', num_to_label)
# Add the decoded labels to the DataFrame
print(df_concat)

In [ ]:
cols = list(df_concat.columns)

#  Move the last column to the third position
# Remove the last column and insert it at the third position (index 2)
last_col = cols.pop(-1)
cols.insert(2, last_col)

# Reindex the DataFrame with the new column order
df_concat = df_concat[cols]
print(df_concat)

In [ ]:
df = decode_labels(df, 'condition_text', num_to_label)
print(df)

In [ ]:
df_temp = pd.concat([df, df_concat], axis=0)
print(df_temp)

In [ ]:
df_sorted = df_temp.sort_values(by='day_id')
print(df_sorted)

In [ ]:
df_fin=df_sorted[["day_id","condition_text"]]
df_fin = df_fin.reset_index(drop=True)
print(df_fin.head)

In [ ]:
df_fin.head()

In [ ]:
df_fin.to_csv("prediction_01.csv", index=False)